# Код Хаффмана

In [4]:
BIG_FILE = 'big_file.txt'
MIDDLE_FILE = 'file.txt'
SMALL_FILE = 'small_file.txt'

curr_file = BIG_FILE

with open(curr_file, encoding='utf-8', mode='r') as file:
    text = file.read()

text

'Newsgroups: alt.angst.xibo.sex,alt.mud,alt.religion.kibo\nFrom: jfurr@nyx.cs.du.edu (JKF)\nSubject: Xibo\'s Vacation (LONG) [Repost]\nMessage-ID: <1992Dec20.032721.23591@mnemosyne.cs.du.edu>\nKeywords: mud angst xibo tank girl\nReply-To: xibo@fido.econ.arizona.edu\nOrganization: Nyx, Public Access Unix @ U. of Denver Math/CS dept.\nDate: Sun, 20 Dec 92 03:27:21 GMT\nLines: 1380\n\n\nYes, Xibo knows I\'m posting this.  Send comments about the writing to Xibo\nat xibo@fido.econ.arizona.edu.\n\nBy: xibo@fido.econ.arizona.edu (Mr. Coates)\nDate: 9 Apr 92 18:41:49 GMT\nOrganization: Xibological Perimeter\n\n\nSeveral months ago, a few angst-list members were mulling over the idea of\nAngstCon III, where to have it, and when.  It became quickly apparent that if\nwe held it in Minneapolis, then the turnout would be pretty good, since five\nof the members live in that area.  So a few arrangements were made, the date\ndebated wildly, and eventually the weekend of March 27th was decided upon.\n

In [5]:
from collections import Counter


class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def __str__(self):
        return self.left, self.right

    def children(self):
        return self.left, self.right


def huffman_code_tree(node_, bin_string=''):
    if type(node_) is str:
        return {node_: bin_string}

    l, r = node_.children()
    d = dict()
    d.update(huffman_code_tree(l, bin_string + '0'))
    d.update(huffman_code_tree(r, bin_string + '1'))
    return d


def make_tree(nodes):
    while len(nodes) > 1:
        key1, c1 = nodes[-1]
        key2, c2 = nodes[-2]
        nodes = nodes[:-2]
        node_ = NodeTree(key1, key2)
        nodes.append((node_, c1 + c2))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    return nodes[0][0]

In [6]:
import pandas as pd

freq = sorted(
    dict(Counter(text)).items(),
    key=lambda kv: kv[1],
    reverse=True,
)

node = make_tree(freq)
encoding = huffman_code_tree(
    make_tree(freq)
)

letter_list = list(encoding.keys())
code_words = list(encoding.values())

pd.DataFrame({
    'Символ': letter_list,
    'Кодовое слово': code_words,
})

,Символ,Кодовое слово
0,,00
1,",",010000
2,f,010001
3,p,010010
4,O,01001100
...,...,...
85,=,1110110110
86,"""",1110110111
87,y,1110111
88,n,11110


## Кодирование текста

In [7]:
coder = {
    c: code_words[i]
    for i, c in enumerate(letter_list)
}

encoded_message = ''.join(
    coder[c]
    for c in text
)

In [8]:
plain_len = len(''.join(format(ord(x), 'b') for x in text))
encoded_len = len(encoded_message)

print(f'Эффективность сжатия: {1 - (encoded_len / plain_len):.3f}')

Эффективность сжатия: 0.292
